# GWASdb-Phenotype

Author: Moshe Silverstein <br/>
Date: 11-17 <br/>
Data Source: http://jjwanglab.org/gwasdb

In [1]:
import sys, datetime, os
import numpy as np
import pandas as pd
import importlib
import untility_functions as uf
%matplotlib inline

In [2]:
importlib.reload(uf)

<module 'untility_functions' from '/Users/moshesilverstein/Documents/Harmonizome/GWASdb/untility_functions.py'>

# Load Data

In [3]:
df = pd.read_csv('Input/gwasdb_20150819_snp_trait.gz', sep='\t')

/Users/moshesilverstein/anaconda/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
df.head()

,CHR,POS,SNPID,REF,ALT,ORI_SNPID,PMID,P_VALUE,P_VALUE_TEXT,OR/BETA,...,MESH_TERM,EFO_ID,EFO_TERM,DOLITE_TERM,RISK_ALLELE,PUBLICATION_TYPE,AA,GENE_SYMBOL,TYPE,REFGENE
0,1,565596,rs568632519,G,A,rs28357983,22091778,7.280000e-05,Taste change in healthy volunteers in response...,NaN,...,NaN,NaN,NaN,Mental retardation,NaN,"Research Support, N.I.H., Extramural",NaN,NaN,NaN,NaN
1,1,807512,rs10751454,A,G,rs10751454,22174011,2.200000e-13,Cardiovascular disease prevalence,NaN,...,NaN,NaN,NaN,Cancer|Cardiovascular disease,NaN,"Research Support, N.I.H., Extramural|Research ...",G,FAM41C,intron,NR_027055
2,1,807512,rs10751454,A,G,rs10751454,22174011,5.400000e-21,Systolic blood pressure (SBP),NaN,...,NaN,NaN,NaN,Cancer|Cardiovascular disease,NaN,"Research Support, N.I.H., Extramural|Research ...",G,FAM41C,intron,NR_027055
3,1,807512,rs10751454,A,G,rs10751454,22174011,7.700000e-09,Total cholesterol,NaN,...,NaN,NaN,NaN,Cancer|Cardiovascular disease,NaN,"Research Support, N.I.H., Extramural|Research ...",G,FAM41C,intron,NR_027055
4,1,880238,rs3748592,A,G,rs3748592,24665060,1.090000e-04,NaN,NaN,...,NaN,NaN,NaN,Drug abuse,NaN,"Research Support, N.I.H., Extramural",G,NOC2L,intron,NM_015658


In [5]:
df.shape

(314237, 30)

# Get Relevent Data

In [6]:
df = df[['GENE_SYMBOL', 'HPO_TERM']]

In [7]:
df.head(10)

,GENE_SYMBOL,HPO_TERM
0,NaN,Cognitive impairment
1,FAM41C,Neoplasm|Abnormality of the cardiovascular system
2,FAM41C,Neoplasm|Abnormality of the cardiovascular system
3,FAM41C,Neoplasm|Abnormality of the cardiovascular system
4,NOC2L,Abnormality of the nervous system
5,NaN,Amyotrophic lateral sclerosis
6,NaN,Growth abnormality
7,C1orf159,Abnormal eating behavior
8,C1orf159,Nausea and vomiting
9,C1orf159,Abnormal eating behavior


In [8]:
df.shape

(314237, 2)

In [9]:
df.drop_duplicates(inplace=True)

In [10]:
df.shape

(61023, 2)

In [11]:
df.dropna(inplace=True)

In [12]:
df.shape

(59614, 2)

In [13]:
df.head()

,GENE_SYMBOL,HPO_TERM
1,FAM41C,Neoplasm|Abnormality of the cardiovascular system
4,NOC2L,Abnormality of the nervous system
7,C1orf159,Abnormal eating behavior
8,C1orf159,Nausea and vomiting
13,TTLL10,Abnormality of the nervous system


In [15]:
df_interactions = pd.DataFrame()

for i, index in enumerate(df.index):
    
    progressPercent = ((i+1)/len(df.index))*100

    sys.stdout.write("Progeres: %d%%  %d Out of %d   \r" % (progressPercent, (i+1), len(df.index)))
    sys.stdout.flush()
    
    if type(df.loc[index, 'GENE_SYMBOL']) != float:
        lst2 = df.loc[index, 'HPO_TERM'].split('|')
        lst1 = [df.loc[index, 'GENE_SYMBOL']]*(len(lst2))
        temp = pd.DataFrame()
        temp['Phenotype'] = lst2
        temp['Gene Name'] = lst1
        df_interactions = pd.concat([df_interactions, temp]) 

In [16]:
df_interactions.head()

,Phenotype,Gene Name
0,Neoplasm,FAM41C
1,Abnormality of the cardiovascular system,FAM41C
0,Abnormality of the nervous system,NOC2L
0,Abnormal eating behavior,C1orf159
0,Nausea and vomiting,C1orf159


In [17]:
df_interactions.shape

(75907, 2)

# Map Gene Symbols To Up-to-date Approved Gene Symbols

In [18]:
df_interactions.set_index('Gene Name', inplace=True)

In [19]:
uf.mapgenesymbols(df_interactions)

# Drop Duplicates

In [20]:
df_interactions.reset_index(inplace=True)

In [21]:
df_interactions.drop_duplicates(inplace=True)

In [22]:
df_interactions.shape

(70482, 2)

# Create Binary Matrix

In [23]:
binary_matrix = uf.createBinaryMatix(df_interactions)

In [24]:
binary_matrix.head()

,Reduced factor VIII activity,Coronary artery stenosis,Neoplasm of the respiratory system,Insomnia,Neoplasm of the gallbladder,Decreased liver function,Abnormality of unsaturated fatty acid metabolism,Nephropathy,Sick sinus syndrome,Abnormality of the calf,...,Abnormality of cholesterol metabolism,Abnormality of the adrenal glands,Neoplasm of the lungs,Abnormal facial shape,Barrett esophagus,Abnormality of body weight,Hyperbilirubinemia,Abnormality of neutrophils,Abnormality of retinal arteries,Increased serum iron
LINC02393,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
PIGL,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ZBTB17,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
IQUB,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
PELI2,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0


In [25]:
binary_matrix.shape

(12652, 487)

# Save Binary Matrix

In [26]:
filename = '~/./Documents/Harmonizome/GWASdb/Output/gwasdb_pheno_binary_matrix_%s.tsv.zip'% str(datetime.date.today())[0:7].replace('-', '_')
binary_matrix.to_csv(filename, sep='\t', compression='gzip')

# Create Gene Set Library

In [27]:
path = '/Users/moshesilverstein/Documents/Harmonizome/GWASdb/Output/'

In [28]:
name = 'gwasdb_pheno_gene_set'

In [29]:
uf.createUpGeneSetLib(binary_matrix, path, name)

# Create Attribute Library

In [30]:
path = '/Users/moshesilverstein/Documents/Harmonizome/GWASdb/Output/'

In [31]:
name = 'gwasdb_pheno_attribute_set'

In [32]:
uf.createUpAttributeSetLib(binary_matrix, path, name)

# Create Gene Similarity Matrix

In [33]:
gene_similarity_matix = uf.createSimilarityMatrix(binary_matrix, 'jaccard')

In [34]:
gene_similarity_matix.head()

,LINC02393,PIGL,ZBTB17,IQUB,PELI2,HRK,ATP6V1G2-DDX39B,ARID1B,ANKRD11,ERVV-2,...,SLC5A9,ARHGAP25,KRT7,CD8A,KCNK13,COL4A5,THRB,PPP1R17,CDC42BPG,KIAA1109
LINC02393,1.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.000000,0.0,0.0,0.200000,0.00,0.000000,0.0,0.0,0.0
PIGL,0.0,1.000000,0.0,0.0,0.083333,0.250000,0.000000,0.086957,0.062500,0.000000,...,0.0,0.181818,0.0,0.0,0.000000,0.00,0.050000,0.0,0.0,0.0
ZBTB17,0.0,0.000000,1.0,0.0,0.200000,0.000000,0.000000,0.043478,0.000000,0.000000,...,0.0,0.000000,0.0,0.0,0.142857,0.25,0.000000,0.0,0.0,0.0
IQUB,0.0,0.000000,0.0,1.0,0.000000,0.000000,0.000000,0.045455,0.153846,0.000000,...,0.0,0.100000,0.0,0.0,0.166667,0.00,0.052632,0.5,0.0,0.0
PELI2,0.0,0.083333,0.2,0.0,1.000000,0.111111,0.071429,0.071429,0.047619,0.111111,...,0.0,0.125000,0.0,0.0,0.076923,0.10,0.068182,0.0,0.0,0.0


# Save Gene Similarity Matrix

In [35]:
filename = '~/./Documents/Harmonizome/GWASdb/Output/gwasdb_pheno_gene_similarity_matix_%s.tsv.zip'% str(datetime.date.today())[0:7].replace('-', '_')
gene_similarity_matix.to_csv(filename, sep='\t', compression='gzip')

# Create Attribute Similarity matrix

In [36]:
attribute_similarity_matix = uf.createSimilarityMatrix(binary_matrix.T, 'jaccard')

In [37]:
attribute_similarity_matix.head()

,Reduced factor VIII activity,Coronary artery stenosis,Neoplasm of the respiratory system,Insomnia,Neoplasm of the gallbladder,Decreased liver function,Abnormality of unsaturated fatty acid metabolism,Nephropathy,Sick sinus syndrome,Abnormality of the calf,...,Abnormality of cholesterol metabolism,Abnormality of the adrenal glands,Neoplasm of the lungs,Abnormal facial shape,Barrett esophagus,Abnormality of body weight,Hyperbilirubinemia,Abnormality of neutrophils,Abnormality of retinal arteries,Increased serum iron
Reduced factor VIII activity,1.0,0.000000,0.000000,0.000000,0.000000,0.034483,0.000000,0.012500,0.000000,0.0,...,0.006281,0.0,0.005420,0.0,0.000000,0.005376,0.000000,0.000000,0.0,0.0
Coronary artery stenosis,0.0,1.000000,0.027778,0.035714,0.027778,0.025641,0.019231,0.000000,0.000000,0.0,...,0.006203,0.0,0.009892,0.0,0.027027,0.000000,0.000000,0.000000,0.0,0.0
Neoplasm of the respiratory system,0.0,0.027778,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,...,0.002500,0.0,0.002700,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0
Insomnia,0.0,0.035714,0.000000,1.000000,0.000000,0.000000,0.000000,0.013699,0.006623,0.0,...,0.000000,0.0,0.002720,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0
Neoplasm of the gallbladder,0.0,0.027778,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.0,...,0.003755,0.0,0.004509,0.0,0.035714,0.010753,0.012048,0.017241,0.0,0.0


# Save Attribute Similarity Matrix

In [38]:
filename = '~/./Documents/Harmonizome/GWASdb/Output/gwasdb_pheno_attribute_similarity_matix_%s.tsv.zip'% str(datetime.date.today())[0:7].replace('-', '_')
attribute_similarity_matix.to_csv(filename, sep='\t', compression='gzip')

# Create Gene List

In [39]:
gene_list = uf.createGeneList(binary_matrix)

In [40]:
gene_list.head()

,GeneSym,GeneID
0,LINC02393,400087
1,PIGL,9487
2,ZBTB17,7709
3,IQUB,154865
4,PELI2,57161


In [41]:
gene_list.shape

(12652, 2)

# Save Gene List

In [42]:
filename = '~/./Documents/Harmonizome/GWASdb/Output/gwasdb_pheno_gene_list_%s.tsv.zip'% str(datetime.date.today())[0:7].replace('-', '_')
gene_list.to_csv(filename, sep='\t', index=False, compression='gzip')

# Create Attribute List

In [43]:
attribute_list = uf.createAttributeList(binary_matrix)

In [44]:
attribute_list.head()

,Attributes
0,Reduced factor VIII activity
1,Coronary artery stenosis
2,Neoplasm of the respiratory system
3,Insomnia
4,Neoplasm of the gallbladder


In [45]:
attribute_list.shape

(487, 1)

# Save Attribute List

In [46]:
filename = '~/./Documents/Harmonizome/GWASdb/Output/gwasdb_pheno_attribute_list_%s.tsv.zip'% str(datetime.date.today())[0:7].replace('-', '_')
attribute_list.to_csv(filename, sep='\t', index=False, compression='gzip')

# Create Gene-Attribute Edge List

In [47]:
path = '/Users/moshesilverstein/Documents/Harmonizome/GWASdb/Output/'

In [48]:
name = 'gwasdb_pheno_gene_attribute_edge_list'

In [49]:
uf.createGeneAttributeEdgeList(binary_matrix, gene_list, path, name)

Progeres: 100%  487 Out of 487   

 The number of statisticaly relevent gene-attribute associations is: 70482
